### Download necessary packages

In [6]:
from importlib.resources import path
from google.cloud import vision
import requests, json, os, io, openai

load the credentials from the json file

In [8]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'macro-outpost-393714-a5aaeaa4013e.json'

openai.api_key = "sk-"

### Authenticate

In [9]:
client = vision.ImageAnnotatorClient()

prepare image

In [10]:
def prepare_image(image_path):
    try:
        with io.open(image_path, 'rb') as image_file:
            content = image_file.read()
        image = vision.Image(content=content)
        return image
    except Exception as e:
        print(e)
        return None

object detection

In [11]:
def detect_object_name(image_path):
    image = prepare_image(image_path)
    objects = client.object_localization(image=image).localized_object_annotations
    if objects:
        result = []
        for object_ in objects:
            result.append(object_.name)
        return result
    else:
        return None

In [12]:
detect_object_name('i.jpeg')[0]

'Box'

Generate the output

In [13]:
def generate_recycling_response(object_name):
    # object_name = detect_object_name(image_path)[0]

    URL = "https://api.openai.com/v1/chat/completions"

    payload = {
        "model": "gpt-3.5-turbo",
        "messages": [{"role": "user", "content": f"Summarize in 50 words the best way to recycle {object_name}?"}],
        "temperature" : 1.0,
        "top_p":1.0,
        "n" : 1,
        "stream": False,
        "presence_penalty":0,
        "frequency_penalty":0,
        }

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {openai.api_key}"
    }

    response = requests.post(URL, headers=headers, json=payload, stream=False)
    # Check if the response status code is 200 (OK)
    if response.status_code == 200:
    # Parse the response JSON
        parsed_response = json.loads(response.text)

    # Extract and beautify the content
        content = parsed_response['choices'][0]['message']['content']
        beautified_response = content.replace("\n\n", "\n")

    # Print the beautified response
        return beautified_response
    else:
        return f"Error: Kindly retake the picture."

In [14]:
object_name = detect_object_name('i.jpeg')[0]
generate_recycling_response(object_name)

'The best way to recycle a box is to first check if it can be reused or repurposed. If not, flatten it to save space and remove any non-recyclable materials. Ensure it is clean and free of contaminants, then place it in the appropriate recycling bin or take it to a recycling center.'